In [13]:
import damask
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml

# 双相钢生成

In [8]:
# Elasticity Part
with open("Hooke_Fe.yaml", 'r', encoding='utf-8') as f:
    martenxite_elasticity = yaml.safe_load(f)

with open("Hooke_Fe.yaml", 'r', encoding='utf-8') as f:
    ferrite_elasticity = yaml.safe_load(f)

with open("Hooke_X2CrNiMo17-12-2.yaml", 'r', encoding='utf-8') as f:
    particle_elasticity = yaml.safe_load(f)

# Plasticity Part
with open("phenopowerlaw_bcc-martensite.yaml", 'r', encoding='utf-8') as f:
    martenxite_plasticity = yaml.safe_load(f)

with open("phenopowerlaw_DP-steel-ferrite.yaml", 'r', encoding='utf-8') as f:
    ferrite_plasticity = yaml.safe_load(f)

with open("kinehardening_X2CrNiMo18-15-4.yaml", 'r', encoding='utf-8') as f:
    particle_plasticity = yaml.safe_load(f)

In [16]:
# create Materials

Material_DP = damask.ConfigMaterial()

homogenization = {
    'SX': {
        'N_constituents': 1,
        'mechanical': {'type': 'pass'}  # No volume fractions, so no homogenization needed
    }
}

Material_DP.update(homogenization=homogenization)

phase_martensite = {
    'Martensite': {
        'lattice': 'cI',
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': martenxite_elasticity,
            'plastic': martenxite_plasticity
        }
    }
}

phase_ferrite = {
    'Ferrite': {
        'lattice': 'cI',
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': ferrite_elasticity,
            'plastic': ferrite_plasticity
        }
    }
}

Material_DP.update(phase=phase_martensite | phase_ferrite)

N_grains_ferrite = 60
rotations = damask.Rotation.from_random(N_grains_ferrite)
Material_DP = Material_DP.material_add(O=rotations, phase='Ferrite', homogenization='SX')

N_grains_martensite = 40
rotations = damask.Rotation.from_random(N_grains_martensite)
Material_DP = Material_DP.material_add(O=rotations, phase='Martensite', homogenization='SX')

In [17]:
Material_DP.is_complete

True

In [18]:
Material_DP.save('Material_DP.yaml')

# 100 多晶 Voronoi Tessellation

In [19]:
cells = [200, 200, 5]
size = np.array(cells) * 1e-5
N_grains = N_grains_ferrite + N_grains_martensite
seeds = damask.seeds.from_random(size, N_grains, cells)
grid = damask.GeomGrid.from_Voronoi_tessellation(cells, size, seeds)
grid.save('grid_DP_100.vtk')

# 单相损伤


In [49]:
Material_damage = damask.ConfigMaterial()

cells = [200, 200, 5]
size = np.array(cells) * 1e-5
N_grains = 20
seeds = damask.seeds.from_random(size, N_grains, cells)
geom_damage = damask.GeomGrid.from_Voronoi_tessellation(cells, size, seeds)
geom_damage.initial_conditions['phi'] = 1.0
geom_damage.save("grid_Steel_100_damage.vti")
with open("anisobrittle_cubic.yaml", 'r', encoding='utf-8') as f:
    damage = yaml.safe_load(f)

In [46]:
homogenization_damage = {
    'SX': {
        'N_constituents': 1,
        'mechanical': {'type': 'pass'},  # No volume fractions, so no homogenization needed
        'damage': {
            'output': ['phi'],
            'type': 'pass'
        }
    }
}

phase_ferrite_damage = {
    'Ferrite': {
        'lattice': 'cI',
        'rho': 7874,
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': ferrite_elasticity,
            'plastic': ferrite_plasticity
        },
        'damage': damage
    }

}

Material_damage.update(phase= phase_ferrite_damage)

N_grains_ferrite = 20
rotations = damask.Rotation.from_random(N_grains_ferrite)
Material_damage = Material_damage.material_add(O=rotations, phase='Ferrite', homogenization='SX')
Material_damage.update(homogenization=homogenization_damage)

Material_damage

homogenization:
  SX:
    N_constituents: 1
    mechanical: {type: pass}
    damage:
      output: [phi]
      type: pass
phase:
  Ferrite:
    lattice: cI
    rho: 7874
    mechanical:
      output: [F, P, F_e, L_p, O]
      elastic:
        type: Hooke
        references: ["D.J. Dever, Journal of Applied Physics 43(8):3293-3301, 1972, https://doi.org/10.1063/1.1661710, fit to Tab. II (T_min=25\xBAC, T_max=880\xBAC)"]
        C_11: 232200000000.0
        C_11,T: -46830000.0
        C_11,T^2: -59880.0
        C_12: 136400000000.0
        C_12,T: -19700000.0
        C_12,T^2: 3760.0
        C_44: 117000000000.0
        C_44,T: -20150000.0
        C_44,T^2: -748.5
        T_ref: 293.15
      plastic:
        type: phenopowerlaw
        references: ['C.C. Tasan et al., Acta Materialia 81:386-400, 2014, https://doi.org/10.1016/j.actamat.2014.07.071, Table 1', "U.F. Kocks, Metallurgical and Materials Transactions B 1:1121\u20131143, 1970, https://doi.org/10.1007/BF02900224"]
        output:

In [47]:
Material_damage.is_complete

True

In [48]:
Material_damage.save("Steel_damage_singla.yaml")

# 双相损伤模型？

In [3]:
Material_damage = damask.ConfigMaterial()
geom_damage = damask.GeomGrid.load("Damage_Steel/Origin/grid_DP_100.vti")
geom_damage

cells:  200 × 200 × 5
size:   0.002 × 0.002 × 5e-05 m³
origin: 0.0   0.0   0.0 m
# materials: 100

In [32]:
geom_damage.initial_conditions['phi'] = 1.0
geom_damage.save("grid_Steel_100_damage.vti")

In [6]:
with open("anisobrittle_cubic.yaml", 'r', encoding='utf-8') as f:
    damage = yaml.safe_load(f)

In [5]:
homogenization_damage = {
    'SX': {
        'N_constituents': 1,
        'mechanical': {'type': 'pass'},  # No volume fractions, so no homogenization needed
        'damage': {
            'output': ['phi'],
            'type': 'pass'
        }
    }
}

Material_damage.update(homogenization=homogenization_damage)

phase_X2CrNiMo_damage = {
    'X2CrNiMo': {
        'lattice': 'cF',
        'rho': 7896.4,
        'mechanical': {
            'output':  ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': particle_elasticity,
            'plastic': particle_plasticity
        },
        'damage': damage
    }
}

phase_ferrite_damage = {
    'Ferrite': {
        'lattice': 'cI',
        'rho': 7874,
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': ferrite_elasticity,
            'plastic': ferrite_plasticity
        },
        'damage': damage
    }

}

Material_damage.update(phase=phase_X2CrNiMo_damage | phase_ferrite_damage)

N_grains_ferrite = 60
rotations = damask.Rotation.from_random(N_grains_ferrite)
Material_damage = Material_damage.material_add(O=rotations, phase='Ferrite', homogenization='SX')

N_grains_martensite = 40
rotations = damask.Rotation.from_random(N_grains_martensite)
Material_damage = Material_damage.material_add(O=rotations, phase='X2CrNiMo', homogenization='SX')

Material_damage

homogenization:
  SX:
    N_constituents: 1
    mechanical: {type: pass}
    damage: {output: '[phi]', type: pass}
phase:
  X2CrNiMo:
    lattice: cF
    rho: 7896.4
    mechanical:
      output: [xi, chi, sgn(gamma), chi_flip, gamma_flip, gamma]
      elastic:
        type: Hooke
        references: ["J.-M. Scherer et al., Materials Science & Engineering A 900:146471, 2024, https://doi.org/10.1016/j.msea.2024.146471, Eq. (6-8) adjusted to T_ref=293.15K (T_min=20\xB0C, T_max=300\xB0C)"]
        C_11: 218300000000.0
        C_11,T: -51300000.0
        C_12: 144800000000.0
        C_12,T: -27700000.0
        C_44: 125400000000.0
        C_44,T: -30800000.0
        T_ref: 293.15
      plastic:
        type: kinehardening
        references: ['J.A. Wollmershauser et al., International Journal of Fatigue 36(1):181-193, 2012, https://doi.org/10.1016/j.ijfatigue.2011.07.008']
        output: [xi, chi, sgn(gamma), chi_flip, gamma_flip, gamma]
        N_sl: [12]
        xi_0: [70000000.0]
     

In [6]:
Material_damage.is_complete

True

In [7]:
Material_damage.save("Steel_damage.yaml")

# isotropic损伤模型

In [28]:
import pandas as pd

Current_model = pd.read_csv("Damage_iso/Cu.csv", sep=" ")
x , y = Current_model['1_pos'].values, Current_model['2_pos'].values

center_x, center_y = (x.max() + x.min())/2, (y.max() + y.min())/2

# 在Center附近生成半径为 1/3 min(x.max()-x.min(), y.max()-y.min()) 的圆形区域
# 也就是对应的相 phase 改为 Hole
radius = min(x.max()-x.min(), y.max()-y.min())/3.0

Hole_indices = ((x - center_x)**2 + (y - center_y)**2) < radius**2
Current_model.loc[Hole_indices, 'phase'] = 'Hole'

Current_model.to_csv("Damage_iso/Cu_hole.csv", sep=" ", index=False)

In [30]:
Model = damask.Table.load("Damage_iso/Cu_hole.csv")
gridfile = damask.GeomGrid.from_table(Model, 'pos', ['phase', 'qu'])
gridfile.origin = [0, 0, 0]
gridfile.size[2] = 0.05

In [35]:
homogenization_damage = {
    'SX': {
        'N_constituents': 1,
        'mechanical': {'type': 'pass'},  # No volume fractions, so no homogenization needed
        'damage': {
            'output': ['phi'],
            'type': 'pass'
        }
    }
}

with open("isotropic_free-surface.yaml", 'r', encoding='utf-8') as f:
    isotropic_plasticity = yaml.safe_load(f)

Hole = {
    'Hole': {
        'lattice': 'cF',
        'rho': 7896.4,
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p'],
            'elastic': {
                'type': 'Hooke',
                'C_11': 0.1e+9,
                'C_12': 0.0,
                'C_44': 0.05e+9
            },
            'plastic': isotropic_plasticity
        },
        'damage': damage
    }
}

phase_ferrite_damage = {
    'Ferrite': {
        'lattice': 'cI',
        'rho': 7874,
        'mechanical': {
            'output': ['F', 'P', 'F_e', 'L_p', 'O'],
            'elastic': ferrite_elasticity,
            'plastic': ferrite_plasticity
        },
        'damage': damage
    }
}

CM = damask.ConfigMaterial.from_table(Model, phase='phase', O='qu', homogenization='homogenization')
CM['phase']['Cu'] = phase_ferrite_damage
CM['phase']['Hole'] = Hole
CM['homogenization']= homogenization_damage

In [36]:
CM.is_complete

True

In [40]:
CM.save("Steel_damage_iso.yaml")
gridfile.initial_conditions['phi'] = 1.0
gridfile.save("Hole_Steel_iso_damage.vti")